In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from scipy import stats
from datetime import timedelta
from collections import Counter
import cx_Oracle
#from warnings import filterwarnings
#filterwarnings('ignore')

#### Data loading and processing

In [2]:
data = pd.read_csv('royal_canin_sp_order_positions_pg_q1_and_q2_04092018.dsv', sep=';', low_memory=False)

In [3]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147594 entries, 0 to 147593
Data columns (total 34 columns):
SCM_N_ORDER_POS_NR              147594 non-null int64
SCM_N_ORDER_NR                  147594 non-null int64
TDT_T_KEY_ORDER_DATE            147594 non-null object
THR_C_KEY_ORDER_HOUR            147594 non-null object
SIT_N_KEY_SITE                  147594 non-null int64
CUS_N_KEY_CUSTOMER              147594 non-null int64
SCM_SALES_BMF                   147594 non-null object
SCM_SALES_CANCEL_BMF            147594 non-null object
SCM_D_FINANCE_CM1               147594 non-null object
SCM_D_FINANCE_CM2               147594 non-null object
SCM_D_ORIG_FINANCE_ORDER_NET    147594 non-null object
SCM_AP_ID                       147594 non-null int64
SCM_AS_ID                       147594 non-null int64
SCM_MASTER_AP_ID                147594 non-null int64
SCM_IS_ZSP_ORDER                147594 non-null int64
SCM_C_ORDER_STATUS              147594 non-null object
CTP_N_KEY_CUSTOME

In [4]:
data.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,TDT_T_KEY_ORDER_DATE,THR_C_KEY_ORDER_HOUR,SIT_N_KEY_SITE,CUS_N_KEY_CUSTOMER,SCM_SALES_BMF,SCM_SALES_CANCEL_BMF,SCM_D_FINANCE_CM1,SCM_D_FINANCE_CM2,...,CZP_EXPIRY_DATE,CZP_ZTQ_ID,CZP_PERCENTAGE,CZP_PRICE,PP_ID,PG_ID,PG4_PARENT_ID,PG3_PARENT_ID,PP_NAME,AP_SHORT_TEXT
0,439129985,102960853,21/01/2018 00:00,13:31,7,10289744,"24,99",0,"5,7729","2,4986",...,26/12/2018 23:59,152,8,"4,9",36702.0,1599.0,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm"
1,441361276,103481965,28/01/2018 00:00,23:39,7,6104684,"24,99",0,"5,6429","1,8617",...,15/11/2018 23:59,152,8,"4,9",36702.0,1599.0,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm"
2,483453172,113725396,28/06/2018 00:00,16:40,7,9282266,"3,49",0,"1,6143","1,0611",...,26/12/2018 23:59,152,8,"4,9",37204.0,914.0,47.0,4.0,Trixie ausziehbarer Spieltunnel für Frettchen,- ø 10 × 19/75 cm
3,463892437,108829030,16/04/2018 00:00,13:53,7,16515999,"7,49",0,"5,1701","4,3297",...,27/06/2018 23:59,152,8,"4,9",18503.0,215.0,47.0,4.0,Kletterbaum für Hamster,- 15 x 14 x 14 cm
4,439950958,103154180,23/01/2018 00:00,21:34,7,2757473,"24,99",0,"5,6429","4,5734",...,23/01/2019 23:59,152,8,"4,9",36702.0,1599.0,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm"


In [5]:
def get_date(x):
    return x.split(' ')[0]

In [6]:
data['TDT_T_KEY_ORDER_DATE'] = data['TDT_T_KEY_ORDER_DATE'].map(get_date)

In [7]:
%run -i cleaning_n_formatting.py

Duplicated rows detected:  1060
Start dropping..
Number of order IDs having more than one SP_START_DATE now:  0


In [8]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146534 entries, 0 to 147593
Data columns (total 40 columns):
SCM_N_ORDER_POS_NR              146534 non-null int64
SCM_N_ORDER_NR                  146534 non-null int64
TDT_T_KEY_ORDER_DATE            146534 non-null datetime64[ns]
THR_C_KEY_ORDER_HOUR            146534 non-null object
SIT_N_KEY_SITE                  146534 non-null int64
CUS_N_KEY_CUSTOMER              146534 non-null int64
SCM_SALES_BMF                   146534 non-null float64
SCM_SALES_CANCEL_BMF            146534 non-null float64
SCM_D_FINANCE_CM1               146534 non-null float64
SCM_D_FINANCE_CM2               146534 non-null float64
SCM_D_ORIG_FINANCE_ORDER_NET    146534 non-null float64
SCM_AP_ID                       146534 non-null int64
SCM_AS_ID                       146534 non-null int64
SCM_MASTER_AP_ID                146534 non-null int64
SCM_IS_ZSP_ORDER                146534 non-null int64
SCM_C_ORDER_STATUS              146534 non-null object
CTP_

In [9]:
data.head(10)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,TDT_T_KEY_ORDER_DATE,THR_C_KEY_ORDER_HOUR,SIT_N_KEY_SITE,CUS_N_KEY_CUSTOMER,SCM_SALES_BMF,SCM_SALES_CANCEL_BMF,SCM_D_FINANCE_CM1,SCM_D_FINANCE_CM2,...,PG4_PARENT_ID,PG3_PARENT_ID,PP_NAME,AP_SHORT_TEXT,ORDER_DATETIME,WEEK_NUMBER,MONTH_NUMBER,FINANCE_ORDER_NET,REL_CM1,REL_CM2
0,439129985,102960853,2018-01-21,13:31,7,10289744,24.99,0.0,5.7729,2.4986,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-01-21 13:31:00,3,1,25.6694,22.489423,9.733769
1,441361276,103481965,2018-01-28,23:39,7,6104684,24.99,0.0,5.6429,1.8617,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-01-28 23:39:00,4,1,26.5951,21.217818,7.000162
2,483453172,113725396,2018-06-28,16:40,7,9282266,3.49,0.0,1.6143,1.0611,...,47.0,4.0,Trixie ausziehbarer Spieltunnel für Frettchen,- ø 10 × 19/75 cm,2018-06-28 16:40:00,26,6,93.4712,1.727056,1.135216
3,463892437,108829030,2018-04-16,13:53,7,16515999,7.49,0.0,5.1701,4.3297,...,47.0,4.0,Kletterbaum für Hamster,- 15 x 14 x 14 cm,2018-04-16 13:53:00,16,4,35.6943,14.484385,12.129948
4,439950958,103154180,2018-01-23,21:34,7,2757473,24.99,0.0,5.6429,4.5734,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-01-23 21:34:00,4,1,40.1983,14.037658,11.377098
5,457019742,107178514,2018-03-22,16:30,7,16455058,7.98,0.0,3.0150,2.5416,...,47.0,4.0,Target Stick,- 1 Stück,2018-03-22 16:30:00,12,3,29.9916,10.052815,8.474373
6,472474517,110988419,2018-05-19,20:15,7,7889866,29.99,0.0,10.2351,8.8613,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-05-19 20:15:00,20,5,65.3636,15.658715,13.556934
7,464272219,108924125,2018-04-17,18:12,7,10476794,29.99,0.0,10.1351,8.3117,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-04-17 18:12:00,16,4,61.5537,16.465460,13.503169
8,452536377,106125471,2018-03-07,15:58,7,4221306,29.99,0.0,10.4251,9.3381,...,47.0,4.0,Kaninchentragetasche mit Freigehege,"- L 50 x B 54,5 x H 32 cm",2018-03-07 15:58:00,10,3,60.8098,17.143783,15.356242
9,447791830,104997420,2018-02-19,20:53,7,18486370,3.99,0.0,1.5075,1.0757,...,47.0,4.0,Target Stick,- 1 Stück,2018-02-19 20:53:00,8,2,53.7769,2.803248,2.000301


In [10]:
#Not equal to 0 means the value being cancelled
data['POS_VALID'] = (data['SCM_SALES_CANCEL_BMF'] == 0) * 1 

In [11]:
data[data['POS_VALID'] == 0][['SCM_SALES_BMF', 'SCM_SALES_CANCEL_BMF', 'SCM_D_ORIG_FINANCE_ORDER_NET']].sample(10)

,SCM_SALES_BMF,SCM_SALES_CANCEL_BMF,SCM_D_ORIG_FINANCE_ORDER_NET
43790,49.9900,41.3100,41.3140
92092,3.9800,3.2900,3.2893
54156,4.4900,3.7100,3.7107
13844,4.9900,4.1200,4.1240
8491,19.4900,16.1100,16.1074
30826,13.9900,11.5600,11.5620
105226,71.9600,14.8700,59.4711
82905,7.9900,6.6000,6.6033
73430,5.2224,4.3178,4.3160
63032,14.9900,12.3900,12.3884


In [12]:
data['TAXES'] = data['SCM_SALES_BMF'] / data['SCM_D_ORIG_FINANCE_ORDER_NET']

In [13]:
data[['SCM_SALES_BMF', 'SCM_D_ORIG_FINANCE_ORDER_NET', 'TAXES']].sample(10)

,SCM_SALES_BMF,SCM_D_ORIG_FINANCE_ORDER_NET,TAXES
121157,-2.50,-2.0661,1.210009
124061,-7.40,-6.1157,1.210000
16237,15.99,13.2149,1.209998
132252,-5.75,-4.7521,1.209991
53925,49.99,41.3140,1.210001
68748,27.99,23.1322,1.210002
14006,10.49,8.6694,1.210003
3775,0.00,0.0000,NaN
57785,96.99,80.1570,1.210000
108871,-4.40,-3.6364,1.209988


#### Gross sales, net sales

In [14]:
# Almost 2M € gross sales
np.round((data['SCM_SALES_BMF'] * data['POS_VALID']).sum(), 2)

1927670.29

In [15]:
# 1.6M € net sales
np.round(sum(data['SCM_D_ORIG_FINANCE_ORDER_NET'] * data['POS_VALID']), 2)

1596605.22

#### Revenue of SP sold

In [16]:
data[data['SCM_MASTER_AP_ID'] == 42225]['CZP_PRICE'].value_counts()

0.00    5642
4.90     730
4.99     311
1.90      10
Name: CZP_PRICE, dtype: int64

In [17]:
np.round(data[data['SCM_MASTER_AP_ID'] == 42225]['CZP_PRICE'].sum(), 2)

5147.89

#### SP discount

In [18]:
# 115K €
np.round(data[data['SCM_MASTER_AP_ID'] == 42715]['SCM_SALES_BMF'].sum(), 2)

-114916.81

#### How would it be in case we had brand specific SP where discounts only apply to Brand product groups?

In [19]:
data.columns

Index(['SCM_N_ORDER_POS_NR', 'SCM_N_ORDER_NR', 'TDT_T_KEY_ORDER_DATE',
       'THR_C_KEY_ORDER_HOUR', 'SIT_N_KEY_SITE', 'CUS_N_KEY_CUSTOMER',
       'SCM_SALES_BMF', 'SCM_SALES_CANCEL_BMF', 'SCM_D_FINANCE_CM1',
       'SCM_D_FINANCE_CM2', 'SCM_D_ORIG_FINANCE_ORDER_NET', 'SCM_AP_ID',
       'SCM_AS_ID', 'SCM_MASTER_AP_ID', 'SCM_IS_ZSP_ORDER',
       'SCM_C_ORDER_STATUS', 'CTP_N_KEY_CUSTOMER_TYPE', 'SCM_OCI_LAND',
       'OSP_KO_COUNT_NET', 'OSP_3_LEAD_P_PG_ID', 'OSP_3_LEAD_P_PG_SHARE',
       'OSP_4_LEAD_P_PG_ID', 'OSP_4_LEAD_P_PG_SHARE', 'CZP_START_DATE',
       'CZP_EXPIRY_DATE', 'CZP_ZTQ_ID', 'CZP_PERCENTAGE', 'CZP_PRICE', 'PP_ID',
       'PG_ID', 'PG4_PARENT_ID', 'PG3_PARENT_ID', 'PP_NAME', 'AP_SHORT_TEXT',
       'ORDER_DATETIME', 'WEEK_NUMBER', 'MONTH_NUMBER', 'FINANCE_ORDER_NET',
       'REL_CM1', 'REL_CM2', 'POS_VALID', 'TAXES'],
      dtype='object')

In [20]:
# Example
data[data['SCM_N_ORDER_NR'] == 108350020][['SCM_MASTER_AP_ID', 'SCM_SALES_BMF', 'SCM_D_FINANCE_CM1', 'SCM_D_FINANCE_CM2',
                                          'SCM_D_ORIG_FINANCE_ORDER_NET', 'PG3_PARENT_ID', 'PG4_PARENT_ID',
                                          'OSP_3_LEAD_P_PG_ID', 'OSP_4_LEAD_P_PG_ID', 'PP_NAME', 'AP_SHORT_TEXT']]

,SCM_MASTER_AP_ID,SCM_SALES_BMF,SCM_D_FINANCE_CM1,SCM_D_FINANCE_CM2,SCM_D_ORIG_FINANCE_ORDER_NET,PG3_PARENT_ID,PG4_PARENT_ID,OSP_3_LEAD_P_PG_ID,OSP_4_LEAD_P_PG_ID,PP_NAME,AP_SHORT_TEXT
27772,23170,0.0000,0.0000,0.0000,0.0000,2430.0,9272.0,252,261925,Extra Bonuspunkte,- 100 Bonuspunkte extra
64427,6855,3.4900,1.5943,1.2422,2.8843,34.0,1559.0,252,261925,Trixie Katzenmalz,- 100 g
68935,54808,49.9900,6.6440,1.7810,41.3140,252.0,261925.0,252,261925,Royal Canin Maine Coon 31,- Maine Coon Adult 10 + 2 kg
74732,54607,0.0000,-0.3500,-0.6970,0.0000,3.0,34.0,252,261925,Catessy Knabber-Snack 65 g,Mit Geflügel & Käse
74733,65409,1.5548,0.4218,0.0658,1.2850,3.0,34.0,252,261925,Smilla Multi-Vitamin Katzenpaste,- 50 g
74734,65409,1.4352,0.3893,0.0607,1.1861,3.0,34.0,252,261925,Smilla Malt Katzenpaste,- 50 g
88366,79327,28.9900,8.8187,3.9976,23.9587,3.0,301.0,252,261925,PeeWee Wood Pellets,- 9kg
93569,32119,3.9900,1.0375,0.5292,3.2975,2068.0,2072.0,252,261925,Megapack Gourmet Perle 8 x 85 g,- Erlesene Streifen mit Gemüse
93570,69053,1.9900,0.4746,0.0662,1.6446,2068.0,2072.0,252,261925,MP/Megapack - Gourmet 4 x 85 g,- Duetto die Mare Fisch
101800,62414,2.9900,1.3911,1.0165,2.4711,36.0,2173.0,252,261925,Trixie Catnip-Spielspray,- 175 ml


+ Reverse the discount applying and get new sales value per order

In [21]:
data[data['SCM_N_ORDER_NR'] == 108350020]['SCM_SALES_BMF'][:11].sum()

94.42999999999999

In [22]:
data[data['SCM_N_ORDER_NR'] == 108350020]['SCM_SALES_BMF'][:12].sum()

89.71

In [23]:
data_without_sp_disc = data[~(data['SCM_MASTER_AP_ID'] == 42715)]

In [24]:
without_sp_disc = data_without_sp_disc.copy()

In [25]:
# Total sales value if there was no SP discount applying on these orders
np.round(sum(without_sp_disc['SCM_SALES_BMF'] * without_sp_disc['POS_VALID']), 2)

2042587.1

+ Locate the suitable products for the specific brand SP

In [26]:
query = """select PPR_N_PHY_PRODUCT_ID
    , PPR_N_PHY_ARTICLE_ID
    , PPR_V_PRODUCT_DESC
    , PPR_V_LOGISTICS_DESC
    , PPR_BRAND
from zoocube.DDSTD_PPR_PHYS_PRODUCT
where PPR_BRAND like '%oyal%'"""

In [27]:
conn = cx_Oracle.connect('zooor', 'zoo12or', 'zoorep', threaded=True, encoding = "UTF-8", nencoding = "UTF-8")
curs = conn.cursor()
pg4ids = pd.read_sql(query, con = conn)
conn.close()

In [28]:
pg4ids

,PPR_N_PHY_PRODUCT_ID,PPR_N_PHY_ARTICLE_ID,PPR_V_PRODUCT_DESC,PPR_V_LOGISTICS_DESC,PPR_BRAND
0,4892,4528,4892 - Royal Canin Medium Puppy,4528 - Royal Cani- 15 kg ARTICLE IS D,Royal Canin Size
1,4892,4590,4892 - Royal Canin Medium Puppy,"4590 - Royal Cani- 7,5 kg ARTICLE IS",Royal Canin Size
2,4923,22519,4923 - Royal Canin Mini Mature +8,22519 - Royal Canin Mini Mature - 8 kg,Royal Canin Size
3,4923,10110,4923 - Royal Canin Mini Mature +8,10110 - Royal Canin Mini Mature - 2 kg,Royal Canin Size
4,4923,12708,4923 - Royal Canin Mini Mature +8,"12708 - Royal Canin Mini Matur- 7,5 kg",Royal Canin Size
5,7625,22520,7625 - Royal Canin Mini Digestive Care,"22520 - Royal Canin Mini Diges- 9,5 kg",Royal Canin Size
6,7625,13049,7625 - Royal Canin Mini Digestive Care,13049 - Royal Canin Mini Digesti- 3 kg,Royal Canin Size
7,18120,24038,18120 - Welpenset (ohne Futter),24038 - Welpenset (ohne Futter)- Maxi,Royal Canin Size
8,18120,24040,18120 - Welpenset (ohne Futter),24040 - Welpenset (ohne Futter)- Mini,Royal Canin Size
9,18120,23695,18120 - Welpenset (ohne Futter),23695 - Welpenset (ohne Futter)- Giant,Royal Canin Size


In [29]:
rc_prods = list(set(pg4ids['PPR_N_PHY_ARTICLE_ID']))

In [30]:
rc_names = list(without_sp_disc[without_sp_disc['SCM_AP_ID'].isin(rc_prods)]['PP_NAME'].unique())

In [31]:
len(rc_names)

185

In [32]:
rc_names

['Royal Canin Nutritional Supplement',
 'Royal Canin - Vet Diet Hypoallergenic',
 'Royal Canin Veterinary Diet Dog - Sensitive Control',
 'Royal Canin Vet Diet - Urinary S/O',
 'RC Veterinary Diet Gastro Intestinal Junior',
 'Royal Canin Vet Diet - Satiety Support weight management',
 'Royal Canin Veterinary Diet Dog - Gastro Intestinal',
 'Royal Canin Veterinary Diet Dog - Mobility C2P+',
 'Royal Canin Veterinary Diet Anallergenic',
 'RC Veterinary Diet Skin Care Small',
 'RC Veterinary Urinary S/O MC',
 'Royal Canin Medium Light Weight Care',
 'RC Veterinary Diet Hypoallergenic Small',
 'RC Veterinary Diet Hypoallergenic MC',
 'Royal Canin Veterinary Diet Dog - Hepatic',
 'RC Veterinary Diet Gastro Intestinal Low Fat',
 'Royal Canin Veterinary Diet Dog - Obesity Mangement',
 'Royal Canin Veterinary Diet Fibre Response',
 'RC Veterinary Diet Skin Care',
 'Royal Canin Veterinary Diet Dental Special Small',
 'RC Veterinary Diet Satiety Small',
 'Royal Canin Veterinary Diet Dog - Dental'

In [33]:
# List every product group from Royal Canin...
#rc_prod_groups = list(without_sp_disc[without_sp_disc['PP_NAME'].fillna('-').str.contains('Royal Canin')]['PP_NAME'].unique()) +\
#                 list(without_sp_disc[without_sp_disc['PP_NAME'].fillna('-').str.contains('RC')]['PP_NAME'].unique())

In [34]:
#len(rc_prod_groups)

In [35]:
#test_included_rc_prods = [i for i in rc_prod_groups if 'Vet' not in i ]

In [36]:
#len(test_included_rc_prods)

In [37]:
# Let's exclude from the list all products belonging to Veterinary Diets
included_rc_prods = [i for i in rc_names  if 'Vet' not in i ]

In [38]:
len(included_rc_prods)

133

In [39]:
without_sp_disc['DISC_POS'] = (without_sp_disc['PP_NAME'].isin(included_rc_prods)) * 1

In [40]:
without_sp_disc[['PP_NAME', 'SCM_SALES_BMF', 'DISC_POS', 'CZP_PERCENTAGE']].sample(20)

,PP_NAME,SCM_SALES_BMF,DISC_POS,CZP_PERCENTAGE
55335,Royal Canin Kitten Sterilised,42.9900,1,5
100497,Advantage CAT (NL),32.9900,0,5
1264,Catit Design Senses Massage-Center,0.0000,0,8
18680,Biokat's Classic 3in1,7.9933,0,5
70345,Royal Canin British Shorthair 34,27.9900,1,5
93015,Megapack Gourmet Perle 8 x 85 g,0.0000,0,5
13484,Royal Canin - Vet Diet - Satiety Support,64.9900,0,5
118999,NaN,-0.3500,0,5
82714,Catit Water Fountain Trinkbrunnen 3 l inkl. Fu...,30.9933,0,5
29473,Savic Bag it Up Litter Tray Bags,5.9800,0,8


+ Apply SP discount only on the suitable order positions

In [41]:
without_sp_disc['NEW_SALES_VALUE'] = without_sp_disc['SCM_SALES_BMF'] - \
((without_sp_disc['SCM_SALES_BMF'] * without_sp_disc['CZP_PERCENTAGE'] / 100) * without_sp_disc['DISC_POS'])

In [42]:
without_sp_disc[['PP_NAME', 'SCM_SALES_BMF', 'DISC_POS', 'CZP_PERCENTAGE', 'NEW_SALES_VALUE']].sample(20)

,PP_NAME,SCM_SALES_BMF,DISC_POS,CZP_PERCENTAGE,NEW_SALES_VALUE
103360,"Royal Canin Pouches, 85 g",34.990,1,5,33.2405
24913,Savings Plan,0.000,0,5,0.0000
33340,Sheba Megapack 12x85g,4.990,0,5,4.9900
28794,toothbrush holder zooplus,0.000,0,5,0.0000
19100,Biokats Micro Fresh Katzenstreu,14.990,0,5,14.9900
2324,LED Pointer Catch the Light,2.490,0,5,2.4900
63850,Fellpflege Handschuh,0.000,0,5,0.0000
111381,NaN,0.990,0,8,0.9900
99023,Felix Snacks 60g,0.000,0,5,0.0000
40843,Litter Locker II,6.990,0,5,6.9900


In [43]:
without_sp_disc['BASKET_VALUE'] = (without_sp_disc['SCM_SALES_BMF'] * without_sp_disc['POS_VALID']).groupby(without_sp_disc['SCM_N_ORDER_NR']).transform('sum')

In [44]:
without_sp_disc['O_POS_SHARE'] = (without_sp_disc['SCM_SALES_BMF'] * without_sp_disc['POS_VALID']) / without_sp_disc['BASKET_VALUE']

In [45]:
data[data['SCM_N_ORDER_NR'] == 101550829]

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,TDT_T_KEY_ORDER_DATE,THR_C_KEY_ORDER_HOUR,SIT_N_KEY_SITE,CUS_N_KEY_CUSTOMER,SCM_SALES_BMF,SCM_SALES_CANCEL_BMF,SCM_D_FINANCE_CM1,SCM_D_FINANCE_CM2,...,PP_NAME,AP_SHORT_TEXT,ORDER_DATETIME,WEEK_NUMBER,MONTH_NUMBER,FINANCE_ORDER_NET,REL_CM1,REL_CM2,POS_VALID,TAXES
53988,433251159,101550829,2018-01-01,02:49,7,10197386,49.99,0.0,7.9940,2.9740,...,Royal Canin Sensible 33,- 10 kg,2018-01-01 02:49:00,1,1,39.2479,20.367969,7.577475,1,1.210001
132801,433251160,101550829,2018-01-01,02:49,7,10197386,-2.50,0.0,-2.0661,-2.0661,...,NaN,NaN,2018-01-01 02:49:00,1,1,39.2479,-5.264231,-5.264231,1,1.210009


#### With new brand specific scenario...

##### Gross sales, net sales

In [46]:
# Almost 2M € gross sales
np.round(sum(without_sp_disc['NEW_SALES_VALUE'] * without_sp_disc['POS_VALID']), 2)

1981221.14

In [47]:
# 1.6M € net sales
np.round((without_sp_disc['NEW_SALES_VALUE'] / without_sp_disc['TAXES'] * without_sp_disc['POS_VALID']).sum(), 2)

1641065.68

##### Revenue of SP sold

In [48]:
without_sp_disc[without_sp_disc['SCM_MASTER_AP_ID'] == 42225]['CZP_PRICE'].value_counts()

0.00    5642
4.90     730
4.99     311
1.90      10
Name: CZP_PRICE, dtype: int64

In [49]:
np.round(without_sp_disc[without_sp_disc['SCM_MASTER_AP_ID'] == 42225]['CZP_PRICE'].sum(), 2)

5147.89

##### SP discount

In [50]:
# 62K €
np.round((without_sp_disc['SCM_SALES_BMF'] - without_sp_disc['NEW_SALES_VALUE']).sum(), 2)

61566.7

|  | Gross sales | Net sales | Revenue of SP sold | SP discount |
| --- | --- | --- | --- |
| OLD RC SP | 1927670 € | 1596605 € | 5148 € | 114917 € | 
| Brand-specific RC SP | 1981221 € | 1641066 € | 5148 € | 61567 € |
|Delta| 2.78% | 2.78% | 0% | -46.42% |

Deltas have been calculated like this: (new state - previous state) / previous state * 100

In [51]:
# Orders including Brand Specific Products 
without_sp_disc[(without_sp_disc['DISC_POS'] == 1) & (without_sp_disc['POS_VALID'] == 1)]['SCM_N_ORDER_NR'].nunique()

22212

In [52]:
# All orders
without_sp_disc['SCM_N_ORDER_NR'].nunique()

33072

In [53]:
# % from Orders having Brand specific products
np.round(without_sp_disc[(without_sp_disc['DISC_POS'] == 1) & (without_sp_disc['POS_VALID'] == 1)]['SCM_N_ORDER_NR'].nunique() /\
without_sp_disc['SCM_N_ORDER_NR'].nunique() * 100, 2)

67.16